
# GAN Project - SoAI - WK6 - Navneet Dubey
---


This project is a Generative Adversarial Network (GAN) implementation using PyTorch, specifically designed to generate face images using the CelebA (Celebrity Faces) dataset. The goal is to train a neural network that can generate realistic-looking human face images that don't actually exist, demonstrating the power of generative machine learning techniques.

In [ ]:
!pip install torch torchvision

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!kaggle datasets download -d jessicali9530/celeba-dataset

In [ ]:
!unzip celeba-dataset.zip -d data

## Importing & Initializing

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

# Hyperparameters
batch_size = 128
image_size = 64
nz = 100  # Latent vector size
ngf = 64  # Generator feature map size
ndf = 64  # Discriminator feature map size
num_epochs = 5
lr = 0.0002
beta1 = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Loading dataset
dataset = dset.ImageFolder(root="data",
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

## Generator

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

## Discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

### Custom weights initialization

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
netG = Generator().to(device)
netD = Discriminator().to(device)

# Initialize weights
netG.apply(weights_init)
netD.apply(weights_init)

## Loss & Optimizatation

In [ ]:
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

## Traning the model

In [ ]:
import os

# Create directory if it does not exist
os.makedirs("results", exist_ok=True)

# Training loop
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):
        # Train Discriminator
        netD.zero_grad()
        real_data = data[0].to(device)
        b_size = real_data.size(0)
        label = torch.full((b_size,), 1., device=device)
        output = netD(real_data).view(-1)
        errD_real = criterion(output, label)
        errD_real.backward()

        noise = torch.randn(b_size, nz, 1, 1, device=device)
        fake_data = netG(noise)
        label.fill_(0.)
        output = netD(fake_data.detach()).view(-1)
        errD_fake = criterion(output, label)
        errD_fake.backward()
        optimizerD.step()

        # Train Generator
        netG.zero_grad()
        label.fill_(1.)
        output = netD(fake_data).view(-1)
        errG = criterion(output, label)
        errG.backward()
        optimizerG.step()

        # Print stats and save images
        if i % 100 == 0:
            print(f'[{epoch}/{num_epochs}][{i}/{len(dataloader)}] '
                  f'Loss_D: {errD_real.item() + errD_fake.item()} Loss_G: {errG.item()}')
            vutils.save_image(real_data, f'results/real_samples.png', normalize=True)
            fake = netG(torch.randn(b_size, nz, 1, 1, device=device))
            vutils.save_image(fake.detach(), f'results/fake_samples_epoch_{epoch}.png', normalize=True)


In [ ]:
torch.save(netG.state_dict(), 'generator.pth')
torch.save(netD.state_dict(), 'discriminator.pth')

In [ ]:
!pip install opencv-python

In [ ]:
import cv2
import os
from glob import glob

# Define parameters
image_folder = 'results'  # Folder where interpolated images are saved
video_name = 'gan_interpolation_video.mp4'
frame_rate = 3  # Frames per second

# Get image paths
images = sorted(glob(os.path.join(image_folder, 'fake_samples_epoch_*.png')))

# Load the first image to get dimensions
frame = cv2.imread(images[0])
height, width, layers = frame.shape

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 format
video = cv2.VideoWriter(video_name, fourcc, frame_rate, (width, height))

# Add each image to the video
for image in images:
    video.write(cv2.imread(image))

# Release the video writer
video.release()

print(f"Video saved as {video_name}")
